In [4]:
%%capture
!pip install transformers
import pandas as pd
from transformers import pipeline
disease_df = pd.read_excel('/content/diseases.xlsx')
# Initialize the fill-mask pipeline
unmasker = pipeline('fill-mask', model='microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext')

In [10]:
import pandas as pd

# Set the number of rows to read
num_rows = int(pd.read_excel('/content/diseases.xlsx').shape[0] * 0.01)

# Read the first 1% of rows
disease_df = pd.read_excel('/content/diseases.xlsx', nrows=num_rows)

# Print the first few rows to verify the data was loaded correctly
print(disease_df.head())

                                             disease
0                                            Cholera
1  Cholera due to Vibrio cholerae 01, biovar chol...
2    Cholera due to Vibrio cholerae 01, biovar eltor
3                               Cholera, unspecified
4                     Typhoid and paratyphoid fevers


In [ ]:
import psutil
from tqdm import tqdm

# Create a list of all possible disease names
disease_list = disease_df['disease']

# Initialize an empty dictionary to store the filled-in sentences
sentences_dict = {}

# Calculate the total number of iterations needed for the progress bar
total_iterations = len(disease_list) * 10

# Initialize the progress bar
pbar = tqdm(total=total_iterations)

# Generate filled-in sentences for all diseases
for disease in disease_list:
    sentence = f"[MASK] is medication used in treatment of {disease}"
    filled_sentences = unmasker(sentence, top_k=10)
    if disease not in sentences_dict:
        sentences_dict[disease] = {f"Drug {j+1}": "" for j in range(10)}
    for i in range(len(filled_sentences)):
        if i >= 10:
            break
        if sentences_dict[disease][f"Drug {i+1}"] == "":
            drug_name = filled_sentences[i]['token_str']
            probability = filled_sentences[i]['score']
            sentences_dict[disease][f"Drug {i+1}"] = f"{drug_name} ({probability})"
        # Update the progress bar
        pbar.update(1)
        
        # Check memory usage and print a warning if RAM usage exceeds 90%
        ram_usage = psutil.virtual_memory().percent
        if ram_usage > 90:
            print("Warning: RAM usage exceeds 90%")

# Create a new dataframe from the dictionary of filled-in sentences
result_df = pd.DataFrame(sentences_dict).transpose()

# Print the resulting dataframe
print(result_df)

# Close the progress bar
pbar.close()


100%|██████████| 416390/416390 [4:34:36<00:00, 25.27it/s]

                                                                                 Drug 1  \
Cholera                                                        it (0.07614026218652725)   
Cholera due to Vibrio cholerae 01, biovar cholerae  ciprofloxacin (0.21914391219615936)   
Cholera due to Vibrio cholerae 01, biovar eltor                it (0.40279489755630493)   
Cholera, unspecified                                            c (0.11721079796552658)   
Typhoid and paratyphoid fevers                                 it (0.32757818698883057)   
...                                                                                 ...   
Dependence on kidney dialysis                            dialysis (0.08694919943809509)   
Dependence on wheelchair                                 levodopa (0.13450677692890167)   
Dependence on artificial heart                                 it (0.12971197068691254)   
Dependence on other enabling machines and devices        nicotine (0.13606837391853333)   

In [ ]:
result_df.to_excel('output.xlsx')

In [ ]:
from google.colab import files
files.download('output.xlsx')

In [ ]:
import pandas as pd
drugs_df = pd.read_excel('/content/output (6).xlsx')

In [ ]:
drugs_df = df.filter(like='Drug')
drugs_df = drugs_df.applymap(lambda x: x.split()[0])

In [ ]:
drugs_df

,Unnamed: 0,Drug 1,Drug 2,Drug 3,Drug 4,Drug 5,Drug 6,Drug 7,Drug 8,Drug 9,Drug 10
0,Cholera,it,antibiotic,metronidazole,there,this,medication,azithromycin,drug,antibiotics,steroid
1,"Cholera due to Vibrio cholerae 01, biovar chol...",ciprofloxacin,it,ceftriaxone,azithromycin,doxycycline,there,colistin,this,metronidazole,ampicillin
2,"Cholera due to Vibrio cholerae 01, biovar eltor",it,ciprofloxacin,ceftriaxone,this,azithromycin,there,doxycycline,metronidazole,amoxicillin,colistin
3,"Cholera, unspecified",c,*,this,a,other,b,ct,it,‡,x
4,Typhoid and paratyphoid fevers,it,doxycycline,tetracycline,ciprofloxacin,streptomycin,rifampicin,ceftriaxone,chloramphenicol,azithromycin,penicillin
...,...,...,...,...,...,...,...,...,...,...,...
41350,Dependence on kidney dialysis,dialysis,methadone,it,hemodialysis,naltrexone,dopamine,lithium,buprenorphine,erythropoietin,levodopa
41351,Dependence on wheelchair,levodopa,it,dbs,methadone,medication,this,gabapentin,morphine,warfarin,benzodiazepines
41352,Dependence on artificial heart,it,methadone,dopamine,nicotine,morphine,digoxin,theophylline,this,propranolol,sildenafil
41353,Dependence on other enabling machines and devices,nicotine,methadone,it,medication,this,buprenorphine,morphine,naltrexone,cocaine,levodopa


In [ ]:
import pandas as pd

# read dataframe from Excel file
df = pd.read_excel('/content/output (6).xlsx')

# extract drug names and keep disease column
drugs_df = df.iloc[:, 1:].applymap(lambda x: x.split()[0])
drugs_df.insert(0, df.columns[0], df[df.columns[0]])

# save resulting dataframe to Excel file
drugs_df.to_excel('drug_names.xlsx', index=False)


In [28]:
import psutil
from tqdm import tqdm

# Create a list of all possible disease names
disease_list = disease_df['disease']

# Initialize an empty dictionary to store the filled-in sentences
sentences_dict = {}

# Calculate the total number of iterations needed for the progress bar
total_iterations = len(disease_list) * 10

# Initialize the progress bar
pbar = tqdm(total=total_iterations)

# Generate filled-in sentences for all diseases
for disease in disease_list:
    sentence = f"{disease} is diagnosed by [MASK] test"
    filled_sentences = unmasker(sentence, top_k=10)
    if disease not in sentences_dict:
        sentences_dict[disease] = {f"Drug {j+1}": "" for j in range(10)}
    for i in range(len(filled_sentences)):
        if i >= 10:
            break
        if sentences_dict[disease][f"Drug {i+1}"] == "":
            drug_name = filled_sentences[i]['token_str']
            probability = filled_sentences[i]['score']
            sentences_dict[disease][f"Drug {i+1}"] = f"{drug_name} ({probability})"
        # Update the progress bar
        pbar.update(1)
        
        # Check memory usage and print a warning if RAM usage exceeds 90%
        ram_usage = psutil.virtual_memory().percent
        if ram_usage > 90:
            print("Warning: RAM usage exceeds 90%")

# Create a new dataframe from the dictionary of filled-in sentences
result_df = pd.DataFrame(sentences_dict).transpose()

# Print the resulting dataframe
print(result_df)

# Close the progress bar
pbar.close()


  0%|          | 0/4160 [00:16<?, ?it/s]

100%|██████████| 4160/4160 [01:08<00:00, 60.79it/s]

                                                                               Drug 1  \
Cholera                                                 antigen (0.16780142486095428)   
Cholera due to Vibrio cholerae 01, biovar cholerae      antigen (0.10083886981010437)   
Cholera due to Vibrio cholerae 01, biovar eltor         antigen (0.06473380327224731)   
Cholera, unspecified                                      stool (0.04894469678401947)   
Typhoid and paratyphoid fevers                      serological (0.38781434297561646)   
...                                                                               ...   
Western equine encephalitis                               elisa (0.13130414485931396)   
Eastern equine encephalitis                         serological (0.12067455798387527)   
St Louis encephalitis                                 serologic (0.14505787193775177)   
Australian encephalitis                                  antibody (0.126797154545784)   
California encephalit

In [26]:
result_df

,Drug 1,Drug 2,Drug 3,Drug 4,Drug 5,Drug 6,Drug 7,Drug 8,Drug 9,Drug 10
Cholera,antigen (0.16780142486095428),stool (0.0635145753622055),antibody (0.05925168842077255),serological (0.054609738290309906),elisa (0.054349079728126526),blood (0.037638068199157715),the (0.02441301755607128),serologic (0.02409709244966507),eia (0.02219976857304573),biochemical (0.02037118747830391)
"Cholera due to Vibrio cholerae 01, biovar cholerae",antigen (0.10083886981010437),serological (0.0661403238773346),elisa (0.0479133278131485),culture (0.047901451587677),stool (0.042912013828754425),serologic (0.034243158996105194),biochemical (0.030277837067842484),the (0.029588190838694572),pcr (0.029432222247123718),antibody (0.025645814836025238)
"Cholera due to Vibrio cholerae 01, biovar eltor",antigen (0.06473380327224731),serological (0.05740245804190636),stool (0.04981384798884392),the (0.04653725400567055),elisa (0.037640031427145004),biochemical (0.034152161329984665),cholera (0.0329388864338398),culture (0.03068605437874794),this (0.028690025210380554),serologic (0.02863915078341961)
"Cholera, unspecified",stool (0.04894469678401947),antigen (0.04739835485816002),the (0.044606152921915054),serological (0.037520937621593475),laboratory (0.03693419322371483),blood (0.02804100513458252),serologic (0.02538783848285675),urine (0.02413901872932911),rapid (0.022670000791549683),antibody (0.020034251734614372)
Typhoid and paratyphoid fevers,serological (0.38781434297561646),elisa (0.1322154849767685),serologic (0.10305292904376984),serology (0.03947306051850319),antigen (0.021909033879637718),culture (0.0209583081305027),igm (0.0170352291315794),laboratory (0.016847604885697365),the (0.014447784051299095),biochemical (0.014259308576583862)
...,...,...,...,...,...,...,...,...,...,...
Western equine encephalitis,elisa (0.13130414485931396),antibody (0.12484639137983322),serological (0.12229278683662415),serologic (0.09033037722110748),immunofluorescence (0.05065799504518509),pcr (0.035903677344322205),igm (0.035840969532728195),this (0.02560671605169773),antigen (0.023515978828072548),serology (0.02183667942881584)
Eastern equine encephalitis,serological (0.12067455798387527),antibody (0.11973705887794495),serologic (0.09382418543100357),elisa (0.08885560929775238),pcr (0.05809592083096504),immunofluorescence (0.04249054938554764),antigen (0.02998746745288372),igm (0.028713444247841835),immunofluorescent (0.020578227937221527),clinical (0.019511951133608818)
St Louis encephalitis,serologic (0.14505787193775177),elisa (0.11794673651456833),antibody (0.09021161496639252),serological (0.08520684391260147),pcr (0.03365693613886833),this (0.029515916481614113),igm (0.02860352396965027),the (0.027163265272974968),antigen (0.024705633521080017),immunofluorescence (0.022726383060216904)
Australian encephalitis,antibody (0.126797154545784),elisa (0.1065240129828453),serological (0.09311112016439438),serologic (0.054502591490745544),immunofluorescence (0.03206551820039749),igm (0.029865611344575882),serology (0.026618586853146553),clinical (0.02540917880833149),pcr (0.022151650860905647),this (0.01647805981338024)


In [ ]:
#---------------------------------------------------------------------------------------------------------